In [1]:
import cv2
import mediapipe as mp
import numpy as np
import sounddevice as sd
import threading

# Load grayscale depth map
depth_map_original = cv2.imread("test_depth.png", cv2.IMREAD_GRAYSCALE)
if depth_map_original is None:
    print("❌ Error: Could not load the png. Check the file path.")
    exit()

# Load translucent image for camera overlay
camera_overlay = cv2.imread("test_og.png", cv2.IMREAD_UNCHANGED)
if camera_overlay is None:
    print("❌ Error: Camera overlay missing.")
    camera_overlay = np.zeros_like(depth_map_original)

# Set overlay transparency
overlay_opacity = 0.4  

# Sound settings
bass_freq = 50  
sample_rate = 44100  
volume = 0.10  
fade_speed = 0.12  
texture_variability = 0.5  

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils  
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Start webcam capture
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('/dev/video2')
if not cap.isOpened():
    print("❌ Error: Could not open webcam.")
    exit()

# Get camera frame size
cam_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
cam_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Function to resize depth map while maintaining aspect ratio
def resize_depth_map(depth_map, target_width, target_height):
    h, w = depth_map.shape[:2]
    scale = min(target_width / w, target_height / h)  # Maintain aspect ratio
    new_w, new_h = int(w * scale), int(h * scale)

    resized_depth_map = cv2.resize(depth_map, (new_w, new_h), interpolation=cv2.INTER_NEAREST)

    # Create a blank canvas of target size
    canvas = np.zeros((target_height, target_width), dtype=np.uint8)
    y_offset, x_offset = (target_height - new_h) // 2, (target_width - new_w) // 2
    canvas[y_offset:y_offset + new_h, x_offset:x_offset + new_w] = resized_depth_map

    return canvas

depth_map = resize_depth_map(depth_map_original, cam_width, cam_height)

# Function to get vibration intensity from depth
def get_vibration_intensity(x, y):
    if 0 <= x < cam_width and 0 <= y < cam_height:
        depth_value = depth_map[y, x] / 255.0  
        return depth_value ** 2.5  
    return 0  

# Function to generate smooth sound transitions
def smooth_transition():
    global current_intensity, target_intensity, running
    while running:
        current_intensity += (target_intensity - current_intensity) * fade_speed
        sd.sleep(10)

# Start smooth intensity transition thread
running = True
current_intensity = 0
target_intensity = 0
smoothing_thread = threading.Thread(target=smooth_transition, daemon=True)
smoothing_thread.start()

# Function to generate bass-heavy sound wave
def generate_waveform(intensity, frames):
    t = np.linspace(0, frames / sample_rate, frames, endpoint=False)
    sine_wave = np.sin(2 * np.pi * bass_freq * t)
    noise = np.random.uniform(-1, 1, frames) * 0.02 * texture_variability
    return (sine_wave + noise) * intensity * volume

# Audio callback function
def audio_callback(outdata, frames, time, status):
    global current_intensity
    try:
        if status:
            print("⚠️ Audio Stream Warning:", status)
        wave = generate_waveform(current_intensity, frames) if current_intensity > 0 else np.zeros(frames)
        outdata[:] = np.column_stack((wave, wave))  
    except Exception as e:
        print("❌ Audio error:", e)

# Start audio stream safely
try:
    stream = sd.OutputStream(callback=audio_callback, samplerate=sample_rate, channels=2)
    stream.start()
except Exception as e:
    print("❌ SoundDevice Error:", e)
    stream = None  

# Function to resize overlay while maintaining aspect ratio
def resize_overlay(overlay, target_width, target_height):
    h, w = overlay.shape[:2]
    scale = min(target_width / w, target_height / h)  
    new_w, new_h = int(w * scale), int(h * scale)

    resized_overlay = cv2.resize(overlay, (new_w, new_h), interpolation=cv2.INTER_LINEAR)
    
    if resized_overlay.shape[2] == 3:
        alpha_channel = np.ones((new_h, new_w), dtype=np.uint8) * 255  
        resized_overlay = np.dstack((resized_overlay, alpha_channel))

    canvas = np.zeros((target_height, target_width, 4), dtype=np.uint8)  
    y_offset, x_offset = (target_height - new_h) // 2, (target_width - new_w) // 2
    canvas[y_offset:y_offset + new_h, x_offset:x_offset + new_w] = resized_overlay
    return canvas

cached_overlay = resize_overlay(camera_overlay, cam_width, cam_height)

# Main loop: track finger & control sound
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    depth_map_display = cv2.cvtColor(depth_map, cv2.COLOR_GRAY2BGR)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            index_finger_tip = hand_landmarks.landmark[8]
            x = int(index_finger_tip.x * cam_width)
            y = int(index_finger_tip.y * cam_height)
            x_pos, y_pos = np.clip(x, 0, cam_width - 1), np.clip(y, 0, cam_height - 1)
            target_intensity = get_vibration_intensity(x_pos, y_pos)
            print(target_intensity)
            cv2.circle(depth_map_display, (x_pos, y_pos), 8, (0, 255, 0), -1)
            cv2.circle(frame, (x_pos, y_pos), int(15 + target_intensity * 40), (255, 255, 255), max(1, int(2 + target_intensity * 4)))
    else:
        target_intensity = 0  

    alpha = cached_overlay[:, :, 3] / 255.0
    for c in range(3):
        frame[:, :, c] = (1 - overlay_opacity * alpha) * frame[:, :, c] + (overlay_opacity * alpha) * cached_overlay[:, :, c]

    cv2.imshow("Camera Feed with Hand Detection", frame)
    cv2.imshow("Depth Map", depth_map_display)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

running = False
cap.release()
cv2.destroyAllWindows()
if stream:
    stream.stop()
    stream.close()


I0000 00:00:1745256370.392043   10779 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1745256370.395323   10840 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.1.3-1pop0~1689084530~22.04~0618746), renderer: Mesa Intel(R) Xe Graphics (TGL GT2)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1745256370.417309   10824 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745256370.435347   10829 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745256375.230549   10827 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


0.0
0.0
0.0
0.0
0.0
0.008019621271004697
0.002186282361398815
0.0024432569282657737
0.05174710470288737
0.053421647475329716
0.02575013901794636
0.0024432569282657737
0.024672862179861053
0.0013238257112571667
0.002186282361398815
0.0024432569282657737
0.0024432569282657737
0.002186282361398815
0.0024432569282657737
0.0024432569282657737
0.0019462462364009702
0.0019462462364009702
0.05010445722599638
0.03280446357795494
0.0024432569282657737
0.0030095361932196726
0.002186282361398815
0.002186282361398815
0.015372788152685392
0.0
0.0
0.0453658286662024
0.04691402634486509
0.05010445722599638
0.04849349992525135
0.04849349992525135
0.04849349992525135
0.04691402634486509
0.043848697680428345
0.04090679183056204
0.0367216198751518
0.03408076916670969
0.03280446357795494
0.03155727423341512
0.060442828547762714
0.07196553619175161
0.09630515818302952
0.10622650606220717
0.10879847213864237
0.10879847213864237
0.14566478332868943
0.1615891987632973
0.18915933913537358
0.20393133869836993
0.

In [4]:
# Load dataset
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
import pickle

df = pd.read_csv("color_training_set.csv")

# Extract features (RGB values) and labels (Color names)
X = df[['R', 'G', 'B']].values  # Features
y = df['Label'].values  # Labels

# Train a K-Nearest Neighbors model
knn = KNeighborsClassifier(n_neighbors=3)  # Use n_neighbors=3 for better accuracy
knn.fit(X, y)

# Save the model for later use
with open("color_classifier.pkl", "wb") as f:
    pickle.dump(knn, f)

print("✅ Model saved as color_classifier.pkl")

✅ Model saved as color_classifier.pkl


In [5]:
import shap
import pandas as pd

explainer = shap.Explainer(knn.predict_proba, X)
shap_values = explainer(X)
shap_dfs = []

for class_idx in range(shap_values.values.shape[2]):
    shap_class_values = shap_values.values[:, :, class_idx]
    shap_df = pd.DataFrame(shap_class_values, columns=['SHAP_R', 'SHAP_G', 'SHAP_B'])
    
    shap_df['R'] = X[:, 0]
    shap_df['G'] = X[:, 1]
    shap_df['B'] = X[:, 2]

    shap_df['Predicted_Label'] = knn.predict(X)

    shap_df['Class'] = class_idx
    
    shap_dfs.append(shap_df)

# Concatenate all DataFrames for each class into one final DataFrame
final_shap_df = pd.concat(shap_dfs, ignore_index=True)

final_shap_df.head()


/home/singhasaur/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,SHAP_R,SHAP_G,SHAP_B,R,G,B,Predicted_Label,Class
0,0.076667,-0.293333,0.116667,4,222,10,Green,0
1,0.111111,0.082778,-0.293889,19,17,234,Blue,0
2,-0.270000,0.050000,0.120000,223,35,1,Red,0
3,0.070556,-0.292778,0.122222,48,243,32,Green,0
4,-0.270000,0.050000,0.120000,227,34,39,Red,0


In [ ]:
import shap
import matplotlib.pyplot as plt

# Convert the DataFrame to a format that SHAP understands
shap_values_for_plot = shap.Explanation(
    values=final_shap_df[['SHAP_R', 'SHAP_G', 'SHAP_B']].values, 
    data=final_shap_df[['R', 'G', 'B']].values,
    feature_names=['R', 'G', 'B']
)

# Create a summary plot
shap.summary_plot(shap_values_for_plot, final_shap_df[['R', 'G', 'B']].values)


In [ ]:
import cv2
import pickle
import numpy as np
import pandas as pd

# Load trained KNN model
with open("color_classifier.pkl", "rb") as f:
    knn = pickle.load(f)

# Load dataset to dynamically create color map
df = pd.read_csv("color_training_set.csv")
color_map = {row["Label"]: [row["R"], row["G"], row["B"]] for _, row in df.iterrows()}

# Load the target image
image = cv2.imread("test_og.png")
if image is None:
    print("❌ Error: Could not load image.")
    exit()

# Convert BGR (OpenCV format) to RGB
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Reshape the image into a 2D array of pixels
pixels = image_rgb.reshape(-1, 3)

# Predict colors for each pixel
predicted_labels = knn.predict(pixels)

# Convert predicted labels into an image using the dynamic color map
classified_pixels = np.array([color_map[label] for label in predicted_labels], dtype=np.uint8)

# Reshape to original image size
classified_image = classified_pixels.reshape(image.shape)

# Save the classified image
cv2.imwrite("test_colour.png", cv2.cvtColor(classified_image, cv2.COLOR_RGB2BGR))
print("✅ Classification completed. Saved as classified_image.png")


In [ ]:
import numpy as np
import sounddevice as sd
from scipy import signal

SAMPLE_RATE = 44100

def envelope(wave, attack=0.01, decay=0.2, sustain=0.8, release=0.1):
    n = len(wave)
    a = int(attack * SAMPLE_RATE)
    d = int(decay * SAMPLE_RATE)
    r = int(release * SAMPLE_RATE)
    s = n - (a + d + r)
    env = np.concatenate([
        np.linspace(0, 1, a),                              # attack
        np.linspace(1, sustain, d),                        # decay
        np.ones(s) * sustain,                              # sustain
        np.linspace(sustain, 0, r)                         # release
    ])
    return wave[:len(env)] * env  # Apply envelope

def generate_sound(instrument, frequency=440, duration=1.0, amplitude=0.5):
    t = np.linspace(0, duration, int(SAMPLE_RATE * duration), endpoint=False)

    if instrument == 'flute':
        repeat_interval = 0.5 # total cycle: silence + note
        note_duration = repeat_interval - 0.1
        samples_per_note = int(SAMPLE_RATE * note_duration)
        silence = np.zeros(int(SAMPLE_RATE * 0.1))

        # Generate one flute note chunk
        t_chunk = np.arange(samples_per_note) / SAMPLE_RATE
        note_chunk = np.sin(2 * np.pi * frequency * t_chunk)
        note_chunk = envelope(note_chunk, decay=0.2, sustain=0.6)

        # Combine silence + note
        wave_chunk = np.concatenate([silence, note_chunk])

        # Repeat the full chunk
        num_repeats = int(np.ceil(duration / repeat_interval))
        wave = np.tile(wave_chunk, num_repeats)

        # Trim to exact duration
        wave = wave[:int(SAMPLE_RATE * duration)]

    elif instrument == 'piano':
       
        repeat_interval = 0.5  # seconds
        samples_per_repeat = int(SAMPLE_RATE * repeat_interval)

        # Generate one fixed note chunk
        t_chunk = np.arange(samples_per_repeat) / SAMPLE_RATE
        wave_chunk = (np.sin(2 * np.pi * frequency * t_chunk) +
                    0.5 * np.sin(4 * np.pi * frequency * t_chunk))
        wave_chunk = envelope(wave_chunk, decay=0.15, sustain=0.3)

        # Repeat the exact same waveform
        num_repeats = int(np.ceil(duration / repeat_interval))
        wave = np.tile(wave_chunk, num_repeats)

        # Trim to match the exact duration
        wave = wave[:int(SAMPLE_RATE * duration)]

    elif instrument == 'sax':
        repeat_interval = 0.5  # seconds (includes 0.3s silence)
        note_duration = repeat_interval - 0.1 # actual note length
        samples_per_note = int(SAMPLE_RATE * note_duration)
        silence = np.zeros(int(SAMPLE_RATE * 0.1))

        # Create one sax note chunk
        t_chunk = np.arange(samples_per_note) / SAMPLE_RATE
        modulator = np.sin(2 * np.pi * 2 * frequency * t_chunk)
        note_chunk = np.sin(2 * np.pi * frequency * t_chunk + 5 * modulator)
        note_chunk = envelope(note_chunk, decay=0.25, sustain=0.6)

        # Combine silence + note
        wave_chunk = np.concatenate([silence, note_chunk])

        # Repeat the full chunk
        num_repeats = int(np.ceil(duration / repeat_interval))
        wave = np.tile(wave_chunk, num_repeats)

        # Trim to exact duration
        wave = wave[:int(SAMPLE_RATE * duration)]


    elif instrument == 'guitar':
        N = int(SAMPLE_RATE / frequency)
        output = []
        decay_factor = 1.0
        refresh_interval = 0.5 # Refresh every 5 seconds

        for i in range(int(SAMPLE_RATE * duration)):
            # Reinitialize the buffer every 'refresh_interval' seconds
            if i % (SAMPLE_RATE * refresh_interval) == 0:
                buffer = np.random.uniform(-1, 1, N)

            avg = 0.5 * (buffer[0] + buffer[1])
            output.append(avg)
            buffer = np.append(buffer[1:], [avg * decay_factor])

        wave = np.array(output)
        wave = envelope(wave, attack=0.01, decay=0.05, sustain=1.0, release=0.1)

    elif instrument == 'conga':
        wave = np.zeros_like(t)
        hit_interval = 0.5 # seconds
        hit_samples = int(hit_interval * SAMPLE_RATE)

        # Make a single conga hit
        def single_conga_hit(length):
            hit_t = np.linspace(0, length / SAMPLE_RATE, length, endpoint=False)
            sine = np.sin(2 * np.pi * frequency * hit_t) * np.exp(-6 * hit_t)

            # Slap noise
            noise = np.random.normal(0, 1, length)
            b, a = signal.butter(3, 800 / (0.5 * SAMPLE_RATE), btype='high')
            slap = signal.lfilter(b, a, noise) * np.exp(-10 * hit_t)

            hit = 0.6 * sine + 0.4 * slap
            return envelope(hit, attack=0.001, decay=0.2, sustain=4.0, release=0.1)

        hit_wave = single_conga_hit(int(0.5 * SAMPLE_RATE))  # 0.5s hit

        for i in range(0, len(t), hit_samples):
            end = min(i + len(hit_wave), len(wave))
            wave[i:end] += hit_wave[:end - i]

    else:
        raise ValueError("Unknown instrument")

    wave = amplitude * wave / np.max(np.abs(wave))
    return wave

def play_instrument(instrument, pitch=440, duration=1.0, amplitude=0.5):
    wave = generate_sound(instrument, pitch, duration, amplitude)
    return wave
    # # print(f"Playing {instrument} at {pitch} Hz for with amplitude {amplitude}")
    sd.play(wave, samplerate=SAMPLE_RATE)
    sd.wait()


# play_instrument('flute', pitch=440, duration=3.0, amplitude=1.0)
# play_instrument('piano', pitch=440, duration=3.0, amplitude=1.0)
# play_instrument('guitar', pitch=440, duration=3.0, amplitude=1.0)
# play_instrument('sax', pitch=440, duration=3.0, amplitude=0.7)
# play_instrument('conga', pitch=440, duration=3.0, amplitude=1.0)

#save these sounds as .wav files
from scipy.io import wavfile

def save_sound(wave, filename="output.wav"):
    # Normalize the wave to be in the range of int16 (16-bit WAV)
    wave_int16 = np.int16(wave * 32767)  # Convert to 16-bit PCM format
    wavfile.write(filename, SAMPLE_RATE, wave_int16)
    print(f"Sound saved as {filename}")

save_sound(play_instrument('guitar', pitch=440, duration=5.0, amplitude=1.0), "sounds1/red_guitar.wav")
save_sound(play_instrument('piano', pitch=440, duration=5.0, amplitude=1.0), "sounds1/yellow_piano.wav")
save_sound(play_instrument('flute', pitch=440, duration=5.0, amplitude=1.0), "sounds1/blue_flute.wav")
save_sound(play_instrument('conga', pitch=440, duration=5.0, amplitude=1.0), "sounds1/green_conga.wav")
save_sound(play_instrument('sax', pitch=440, duration=5.0, amplitude=0.6), "sounds1/black_sax.wav")


In [3]:
import cv2
import numpy as np
import mediapipe as mp
import pickle
import sounddevice as sd
import soundfile as sf

# Load the KNN model
with open("color_classifier.pkl", "rb") as f:
    knn = pickle.load(f)

# Load classified image
classified_image = cv2.imread("test_colour.png")
if classified_image is None:
    print("❌ Error: Could not load classified image.")
    exit()

# Load translucent overlay
overlay = cv2.imread("test_og.png", cv2.IMREAD_UNCHANGED)
if overlay is None:
    print("❌ Error: Could not load overlay.")
    overlay = np.zeros_like(classified_image)

# Start camera
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('/dev/video2')

if not cap.isOpened():
    print("❌ Error: Could not open webcam.")
    exit()

# Get camera size
cam_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
cam_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Resize overlay and classified image while keeping aspect ratio
def resize_with_aspect_ratio(image, target_width, target_height):
    """ Resize an image while maintaining its aspect ratio and adding padding. """
    h, w = image.shape[:2]
    aspect_ratio = w / h

    if w > h:
        new_w = target_width
        new_h = int(target_width / aspect_ratio)
    else:
        new_h = target_height
        new_w = int(target_height * aspect_ratio)

    resized = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

    # Add padding to match the target size
    top = (target_height - new_h) // 2
    bottom = target_height - new_h - top
    left = (target_width - new_w) // 2
    right = target_width - new_w - left

    color = [0, 0, 0, 0] if image.shape[2] == 4 else [0, 0, 0]
    return cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)

classified_image = resize_with_aspect_ratio(classified_image, cam_width, cam_height)
overlay = resize_with_aspect_ratio(overlay, cam_width, cam_height)

# Ensure overlay has alpha channel
if overlay.shape[2] == 3:
    alpha_channel = np.ones((cam_height, cam_width), dtype=np.uint8) * 255
    overlay = np.dstack((overlay, alpha_channel))

alpha = overlay[:, :, 3] / 255.0

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

color_sounds = {
    "Red": "/home/singhasaur/code/HCAI/Project_w_sound/FINAL/sounds1/red_guitar.wav",
    "Blue": "/home/singhasaur/code/HCAI/Project_w_sound/FINAL/sounds1/blue_flute.wav",
    "Green": "/home/singhasaur/code/HCAI/Project_w_sound/FINAL/sounds1/green_conga.wav",
    "Yellow": "/home/singhasaur/code/HCAI/Project_w_sound/FINAL/sounds1/yellow_piano.wav",
    "Black": "/home/singhasaur/code/HCAI/Project_w_sound/FINAL/sounds1/black_sax.wav",  
}

# Function to get color classification from classified image
def get_classified_color(x, y):
    bgr = classified_image[y, x]
    rgb = bgr[::-1]  # Convert BGR to RGB
    return knn.predict([rgb])[0]

# Function to play sound (only when color changes)
import threading

last_color = None  # Track last detected color

def play_sound(color):
    global last_color

    # 🛑 Stop all sounds if White is detected
    if color == "White":
        print("🔇 White detected! Stopping all sounds.")
        sd.stop()
        last_color = "White"
        return  # Exit early

    # 🔍 Check if the sound file exists
    if color in color_sounds:
        sound_file = color_sounds[color]
    else:
        print(f"❌ No sound file mapped for {color}")
        return

    # 🛑 Skip if same color as before
    if color == last_color:
        return

    last_color = color  # Update last color

    try:
        data, samplerate = sf.read(sound_file)
        sd.play(data, samplerate, blocking=False)  # ✅ Non-blocking playback
    except Exception as e:
        print(f"❌ Sound error: {e}")

 

# Main loop

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    display_image = classified_image.copy()

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Get index finger tip
            index_finger_tip = hand_landmarks.landmark[8]
            x = int(index_finger_tip.x * cam_width)
            y = int(index_finger_tip.y * cam_height)
            x, y = np.clip(x, 0, cam_width - 1), np.clip(y, 0, cam_height - 1)
            
            # Get color classification
            color_name = get_classified_color(x, y)
            print(f"Detected color: {color_name}")

            # Play sound only when color changes
            play_sound(color_name)

            # Show pointer on classified image
            cv2.circle(display_image, (x, y), 8, (0, 255, 0), -1)
            cv2.circle(frame, (x, y), 8, (255, 255, 255), -1)

    # Apply overlay to camera frame
    for c in range(3):
        frame[:, :, c] = (1 - 0.4 * alpha) * frame[:, :, c] + (0.4 * alpha) * overlay[:, :, c]

    cv2.imshow("Camera Feed with Overlay", frame)
    cv2.imshow("Classified Image", display_image)
    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        sd.stop() 
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1745256440.899578   10779 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1745256440.900390   10919 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.1.3-1pop0~1689084530~22.04~0618746), renderer: Mesa Intel(R) Xe Graphics (TGL GT2)
W0000 00:00:1745256440.919092   10910 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745256440.934450   10915 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected color: Black
Detected c